## RIG plots - with WUSS notation from secondary structure consensus
### background colors are mapped to secondary structure elements of the consensus structure provided by the covariance model

In [1]:
import os
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [2]:
WUSSpath = "../SS_cons/SS_cons_WUSS.tsv"
RIGpath = "../RIG_nogaps/" #gaps removed from the consensus
gapChar = "."

bear90path = RIGpath + "bear_90_RIGs.tsv"
bear50path = RIGpath + "bear_50_RIGs.tsv"
qbear90path = RIGpath + "qbear_90_RIGs.tsv"
qbear50path = RIGpath + "qbear_50_RIGs.tsv"
zbear90path = RIGpath + "zbear_90_RIGs.tsv"
zbear50path = RIGpath + "zbear_90_RIGs.tsv"

In [3]:
from collections import defaultdict


In [4]:
#read WUSS dictionary
##Rremove gaps from the consensus <----
WUSS_dict = OrderedDict()
with open(WUSSpath) as f:
    for line in f:
        #RF\tWUSS
        dat = line.split()
        WUSS_dict[dat[0]] = dat[1].strip().replace(gapChar,"")


In [5]:
#colors WUSS dictionary
WUSS_color_dict = OrderedDict()
stems = "><}{][)("
hairpinloops = "_"
bulgeInterior = "-"
gaps = "."

for k in WUSS_dict:
    colorStr = ""
    for c in WUSS_dict[k]:
        if c in stems:
            colorStr += "r"
        elif c in hairpinloops:
            colorStr += "g"
        elif c in bulgeInterior:
            colorStr += "c"
        elif c in gaps:
            colorStr += "w"
        elif c.isalpha():
            colorStr += "m"
        else:
            colorStr += "b"
    WUSS_color_dict[k] = colorStr


In [6]:
RIG_dict = defaultdict(dict)

def loadRIGs(RIGpath, encoding):
    with open(RIGpath) as f:
        for line in f:
            data = line.split()
            RF = data[0] 
            rig = [float(x) for x in data[1:]]
            RIG_dict[encoding][RF] = rig
    return RIG_dict
        
RIG_dict=loadRIGs(bear90path, 'bear90')
RIG_dict=loadRIGs(bear50path, 'bear50')
RIG_dict=loadRIGs(qbear90path, 'qbear90')
RIG_dict=loadRIGs(qbear50path, 'qbear50')
RIG_dict=loadRIGs(zbear90path, 'zbear90')
RIG_dict=loadRIGs(zbear50path, 'zbear50')



In [7]:
def get_colored_chunks(RIG_values, color_string):
    """assigns color values to the SSE"""
    i = 0
    colored_chunks = []
    while(i < len(color_string)):
        #colore di partenza
        my_color = color_string[i]
        start_idx = i
        while((i < len(color_string)) and (color_string[i] == my_color)):
            i += 1
        end_idx = i
        rangeX = np.arange(start_idx,
                           end_idx ,
                           1)
        #rangeY = RIG_values[start_idx:end_idx]
        rangeY = 1*np.arange(start_idx,end_idx,1)
        
                
            
        rangeX = np.arange(start_idx-0.1, end_idx+0.1)

        #se e' un punto singolo allora disegna una fascia stretta ad altezza punto
        if end_idx - start_idx == 1:
            #rangeX = [start_idx-0.2, start_idx+0.2]
            #rangeY = 2*[RIG_values[start_idx]]
            rangeY = [1,1]
            
        colored_chunks.append([rangeX,
                              rangeY,
                              my_color])
    return colored_chunks


In [8]:
sns.set(context='paper', style='white', palette='deep', font='serif', font_scale=2, color_codes=True, rc=None)

def plot_RIG_WUSS(RF, RIG_dict=RIG_dict, WUSS_color_dict=WUSS_color_dict, fname='test', encs=['bear90']):
    """plots RIG - enhanced with WUSS"""
    #pulisco eventuali prove rimaste appese
    plt.clf()
    
    legend_elements = [ Patch(facecolor='r', label='Stem'),
                  Patch(facecolor='g', label='Hairpin Loop'),
                  Patch(facecolor='c', label='Bulge/Interior'),
                  Patch(facecolor='b', label='Others'),
                  Patch(facecolor='m', label='Pseudoknot')]
    #creo la figura
    fig,ax = plt.subplots(1, figsize=(20,10))
    
    #plotto la RIG
    styles = [ '-', '--', '-.' ,':']
    for enc,st in zip(encs, styles[:len(encs)]):
        ax.plot(RIG_dict[enc][RF],color='k', linewidth=2, ls=st)
        legend_elements.append(
                                Line2D([0], [0],  ls=st, label=enc, color='k'),
        )
    #genero la suddivisione in colori dei vari blocchi
    colored_chunks = get_colored_chunks(RIG_dict[enc][RF], WUSS_color_dict[RF])
    
    #per ogni blocco di colore coloro sotto l'area corrispondente
    for count, colored_chunk in enumerate(colored_chunks):
        plt.fill_between(colored_chunk[0],1, facecolor=colored_chunk[2], alpha=0.5)
    #plt.fill_between(range(len(RIG_dict[enc][RF])), RIG_dict[enc][RF], 
      #               where=list(WUSS_color_dict[RF])=='r',interpolate=True,
      #               facecolor='r', alpha=0.2 )
    ax.set_title(f'-{enc[-2:]}- RIG - {RF}')
    ax.set_xlabel('alignment position (nt)')
    ax.set_ylabel('RIG')

    ax.set_ylim([0,1])
    ##aggiungere legenda
    legend = ax.legend(handles=legend_elements, loc='best', frameon=1)
    frame = legend.get_frame()
    frame.set_facecolor('w')
    
    plt.tight_layout()
    
    #plt.show()
    #

    plt.savefig(fname+".pdf")
    plt.savefig(fname+".png", ppi=600)
    plt.close()


In [9]:
for RF_ in RIG_dict['bear90']:
    print(RF_)
    plot_RIG_WUSS(RF_,
                 encs=['bear90', 'qbear90','zbear90'],
                 fname=f"../RIG_WUSS/{RF_}_90")

RF00001
RF00002
RF00003


KeyboardInterrupt: 